### Практическая работа к уроку № 3

In [87]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import cross_validate

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других

In [36]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

# Так как обучать будем с помощью кросс валидации, то разбивать на train/test не будем
X, y = df.drop('cardio', 1), df['cardio']

In [38]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))


# Объединим все наши трансформеры
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

In [72]:
def train(name, classifier, all_scores):
    print(f"Train {name}")
    scores = cross_validate(classifier, X, y, cv=5,
                            scoring=('roc_auc', 'precision', 'recall', 'f1'),
                            n_jobs=-1)

    scores = {k: v.mean() for k, v in scores.items()}
    
    all_scores.update({name: {k: v for k, v in scores.items() if k.startswith("test_")}})
    
    return all_scores

In [82]:
classifier_1 = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42))
])
classifier_2 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42))
])
classifier_3 = Pipeline([
    ('features',feats),
    ('classifier', KNeighborsClassifier())
])
classifier_4 = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 42))
])
classifier_5 = Pipeline([
    ('features',feats),
    ('classifier', SVC(random_state = 42))
])

classifiers = {"LogisticRegression": classifier_1,
               "RandomForestClassifier": classifier_2,
               "KNeighborsClassifier": classifier_3,
               "GradientBoostingClassifier": classifier_4,
               "SVC": classifier_5}

In [83]:
all_scores = {}
for name_classifier, classifier in classifiers.items():
    all_scores = train(name_classifier, classifier, all_scores)

Train LogisticRegression
Train RandomForestClassifier
Train KNeighborsClassifier
Train GradientBoostingClassifier
Train SVC


In [85]:
result_df = pd.DataFrame.from_dict(all_scores, orient="index")
result_df.sort_values(by=["test_roc_auc"], inplace=True, ascending=False)
result_df

,test_roc_auc,test_precision,test_recall,test_f1
GradientBoostingClassifier,0.802611,0.754593,0.698934,0.725688
SVC,0.789840,0.746546,0.700763,0.722921
LogisticRegression,0.784314,0.742284,0.676434,0.707807
RandomForestClassifier,0.773342,0.718259,0.700392,0.709210
KNeighborsClassifier,0.696247,0.659038,0.631807,0.645132


5. (опциональный вопрос) какой график (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90<br>
вторая помечает 1000 объектов как класс 1, но TP такой же - 90<br>
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

#### Ответ
Посчитаем метрики качества для каждой модели:<br>
    1. precision = 90/100=0.9; recall=90/100=0.9<br>
    2. precision = 90/1000=0.09; recall=90/100=0.9<br>
Очевидно, что первая модель лучше.

Если бы мы строили roc_auc кривую, то результат был бы хороший на обеих моделях,
потому что количество возможных шагов по вертикальной оси зависит от количества объектов класса 1.
А эти объекты в одинаковом количестве распознали обе модели.

Таким образом, в случае дисбаланса классов для оценки качества лучше подойдет precision_recall_curve